# Deep Reinforcement learning

Author : Nicholas Setijabudiharsa

TSE - Econometrics and Statistics : Data science for social science

2022

## 1 Reinforcement learning

### 1.1 Definitions and  Framework

In order to better understand the idea of reinforcement learning, lets take an example of a game of chess. Theoritically speaking, one can create an brute force algorithm that analyze all possible states of the game and takes into account all the possible states that can come after the current state, and take the action that give the most probability of winning at the end. However, this method is very inefficient. In 2021, [John Tromp](https://github.com/tromp/ChessPositionRanking) did an analysis and found that there are about $4.8 x 10^{44}$ possible combinations of legal chess positions in chess. For perspective, here is the full form of that number

$$480000000000000000000000000000000000000000000$$

So for sure, this process is very inefficient. This is where Reinforcement learning comes in handy.



Reinforcement learning is the process optimizing an agent's action $a$ of a given state $s$ by optimizing it's policy $\pi(s, a)$ where
$$
\pi(s, a) = Pr(a=a | s=s)
$$

This allows us to define a notion of Value for a given state for a given policy which will be the expected discounted future rewards or in mathematical terms

$$
V_\pi(s) = E(\sum_t\gamma^t r_t |s_0=s)
$$

with $\gamma$ to be the discount rate.


With our given framework we can now understand that the goal of Reinforcement learning is a **maximization problem** where we **optimize** our agent's policy to **maximize** future rewards.

Generally we also normally assume our model is based on a markovian decision process, where we dont take the next steps with certainty but instead given a current state $s_t$, we assume that $s_{t+1}$ adopts a certain probability distribution. For example, any games that includes rolling a dice.

An environment where we get an information about the reward after every action an agent take, is called **Dense** rewards, on the other hand, environment such as playing a simple game of chess where we only receive rewards at the end of the game (whether we win or loses), is called **Sparse** rewards, naturally, the denser the rewards, the more efficient our data will be and therefore faster our learning will be.

## 2 Deep Reinforcement Learning [draft only](https://www.youtube.com/watch?v=wDVteayWWvU&ab_channel=SteveBrunton)

### 2.1 Deep policy network

Takes input state and returns policy $\pi_\theta(s, a)$ where $\theta$ is weight of our NN

### 2.2 Deep Q Network

## 3 Implementation